In [1]:
import requests
from bs4 import BeautifulSoup

import os
import re
import pickle

from tqdm import tqdm, trange

In [2]:
GENIUS_API_TOKEN='0DRkNDsVx6eWmcAXzWNLct1DPEQ24l79DHvie_0YeGtXfih-pcl6BNO0uBVCIxQm'

In [3]:
# Get artist object from Genius API
def request_artist_info(artist_name, page):
    conn_timeout = 1
    read_timeout = 10
    timeouts = (conn_timeout, read_timeout)
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers, timeout=timeouts)
    return response

# Get Genius.com song url's from artist object
def request_song_url(artist_name, song_cap):
    page = 1
    songs = []
    zero_iter = 0
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()
        # Collect up to song_cap song objects from artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
                
        if not len(song_info):
            zero_iter += 1
        if zero_iter >= 3:
            break
#         print(f'song_info on page {page}: {len(song_info)}')

        # Collect song URL's from song objects
        for song in song_info:
            if (len(songs) < song_cap):
                # print(song['result'])
                title = song['result']['title']
                url = song['result']['url']
                songs.append((title, url))
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    # print('Found {} songs by {}'.format(len(songs), artist_name))
    return songs
    
# DEMO
# request_song_url('j balvin', 2)

In [11]:
# Scrape lyrics from a Genius.com song URL
def scrape_song_lyrics(url):
    conn_timeout = 1
    read_timeout = 10
    timeouts = (conn_timeout, read_timeout)
    page = requests.get(url, timeout=timeouts)
    html = BeautifulSoup(page.text, 'html.parser')
    try:
        lyrics = html.find('div', class_='lyrics').get_text()
        #remove identifiers like chorus, verse, etc
        lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
        #remove empty lines
        lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])      
    except AttributeError:
        lyrics = ''   
    return lyrics

# # DEMO
# scrape_song_lyrics('https://genius.com/Lana-del-rey-young-and-beautiful-lyrics')

In [5]:
def get_lyrics(artist_name, song_count):
    output = []
    res = request_song_url(artist_name, song_count)
    if not res: return []
    
    titles, urls = list(zip(*res))
    for i, url in enumerate(tqdm(urls)):
        lyrics = scrape_song_lyrics(url)
        output.append((titles[i], lyrics))
    
    if not output:
        print(f'no lyrics for {artist_name}')
    return output

In [12]:
new_a = []
with open('new_artists.txt', 'r+') as fp:
    for line in fp:
        new_a.append(line.strip('\n'))


In [13]:
new_a[:5]

['? & The Mysterions',
 'Aaron Neville',
 'Archie Bell & The Drells',
 'Arthur Alexander',
 'Arthur Conley']

In [10]:

index = 128
for artist in new_a[index:]:
    index += 1
    print(index, artist)
    if artist in res:
        continue
    res[artist] = get_lyrics(artist, 30)

0 ? & The Mysterions
0 Aaron Neville
100%|██████████| 30/30 [00:16<00:00,  1.81it/s]
0 Archie Bell & The Drells
100%|██████████| 13/13 [00:16<00:00,  1.29s/it]
0 Arthur Alexander
100%|██████████| 15/15 [00:21<00:00,  1.43s/it]
0 Arthur Conley
100%|██████████| 8/8 [00:07<00:00,  1.01it/s]
0 Barbara Lewis
100%|██████████| 24/24 [00:34<00:00,  1.45s/it]
0 Barbara Lynn
100%|██████████| 11/11 [00:13<00:00,  1.23s/it]
0 Betty Everett
100%|██████████| 11/11 [00:22<00:00,  2.03s/it]
0 Billy Stewart
100%|██████████| 11/11 [00:12<00:00,  1.16s/it]
0 Blood, Sweat & Tears
100%|██████████| 30/30 [00:41<00:00,  1.39s/it]
0 Blue Cheer
100%|██████████| 30/30 [00:44<00:00,  1.50s/it]
0 Bob Dylan
100%|██████████| 30/30 [00:26<00:00,  1.14it/s]
0 Bobby Womack/Valentinos
0 Booker T. & The MG's
0 Buffalo Springfield
100%|██████████| 30/30 [00:43<00:00,  1.44s/it]
0 Canned Heat
100%|██████████| 30/30 [00:37<00:00,  1.26s/it]
0 Cannibal & The Headhunters
100%|██████████| 2/2 [00:02<00:00,  1.32s/it]
0 Captai

AttributeError: 'NoneType' object has no attribute 'get_text'

In [15]:
len(res)

128